In [ ]:
# !pip install requests
# !pip install bs4
# !pip install selenium
# !pip install numpy

In [1]:
from tqdm import tqdm
from urllib.request import Request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import urllib.request
import re
import numpy as np
import time
import os
import random
import time
import pandas as pd

#  투순지 타잔지에 따라 달라짐, https://statiz.sporki.com/player/?m=day&p_no=14788&pos=pitching&year=2021

def player_score_batting(year, p_no):
    url = f'https://statiz.sporki.com/player/?m=day&p_no={p_no}&pos=batting&year={year}'
    req = Request(url,headers={'User-Agent':'Mozila/5.0'})
    webpage = urlopen(req)
    soup = BeautifulSoup(webpage,'html.parser')

    element = soup.find_all(class_ = 'table_type02 transverse_scroll')[0]
    headers = [th['tooltip'] for th in element.find_all('th')]
    headers = headers[:32] # 32개만 header로 존재,

    data = []
    for row in element.find_all('tr'):
        row_data = [td.text for td in row.find_all('td')]
        if row_data:
            data.append(row_data)

    # 데이터프레임 생성
    df = pd.DataFrame(data, columns=headers)

    return df


def player_score_pitching(year, p_no):

    url = f'https://statiz.sporki.com/player/?m=day&p_no={p_no}&pos=pitching&year={year}'

    req = Request(url,headers={'User-Agent':'Mozila/5.0'})
    webpage = urlopen(req)
    soup = BeautifulSoup(webpage,'html.parser')



    element = soup.find_all(class_ = 'table_type02 transverse_scroll')[0]
    #element = soup.select_one("html > body > div:nth-of-type(2) > div:nth-of-type(2) > section > div:nth-of-type(3)")
    headers = [th['tooltip'] for th in element.find_all('th')]
    headers = headers[:30] # 32개만 header로 존재,

    data = []
    for row in element.find_all('tr'):
        row_data = [td.text for td in row.find_all('td')]
        if row_data:
            data.append(row_data)

    # 데이터프레임 생성
    df = pd.DataFrame(data, columns=headers)

    return df

## 신인 선수들 추출
kbo신인상 받은 선수 + 최근 5년안에 데뷔한 선수

In [47]:
rookie = pd.read_csv('/content/kbo_rookie_award.csv')
rookie1 = pd.read_csv('/content/rookie_player_1_year.csv', index_col = 0)
rookie2 = pd.read_csv('/content/rookie_player_2_year.csv',  index_col = 0)

rookie_sta = pd.read_csv('/content/bat_code.csv')

In [48]:
rookie1 = rookie1.merge(rookie_sta[['Name',"rink"]], on = "Name")
rookie2 = rookie2.merge(rookie_sta[['Name',"rink"]], on = "Name")

In [49]:
rookie['position'] = rookie['position'].apply(lambda x : 'pitching' if x == '투수' else 'batting')

In [50]:
rookie.rename(columns = {'year':'award_year'}, inplace= True)

In [51]:
rookie1.rename(columns={'Name': 'name', 'Graduate': 'HS_Graduate',
                        'Year':'debut', 'Team':'club', 'Pos':'position',
                        'rink':'num'}, inplace=True)
rookie2.rename(columns={'Name': 'name', 'Graduate': 'HS_Graduate',
                        'Year':'debut', 'Team':'club', 'Pos':'position',
                        'rink':'num'}, inplace=True)

In [52]:
rookie1['HS_Graduate'] = rookie1['HS_Graduate'].apply(lambda x : "y" if x == '고졸' else "n")
rookie2['HS_Graduate'] = rookie2['HS_Graduate'].apply(lambda x : "y" if x == '고졸' else "n")

In [53]:
rookie1['position'] = rookie1['position'].apply(lambda x :'pitching' if x == 'P' else "batting" )
rookie2['position'] = rookie2['position'].apply(lambda x :'pitching' if x == 'P' else "batting" )

In [54]:
rookie_merge = pd.concat([rookie, rookie1[['name', 'HS_Graduate', 'debut','club','position', 'num']]], axis = 0)
rookie_merge = pd.concat([rookie_merge, rookie2[['name', 'HS_Graduate', 'debut','club','position', 'num']]], axis = 0)
rookie_merge.to_csv('rookie_merge.csv')

In [55]:
rookie_merge.drop_duplicates(subset=['name','debut'], inplace=True) #같은 사람 key만 다르게 두번 들어가는 경우 생김 이를 제거


In [56]:
players = dict.fromkeys(rookie_merge['name'])

In [57]:
for player in players:
    players[player] = {
        'p_no': rookie_merge[rookie_merge['name'] == player]['num'].values[0],
        'pos': rookie_merge[rookie_merge['name'] == player]['position'].values[0],
        'year': [rookie_merge[rookie_merge['name'] == player]['debut'].values[0],rookie_merge[rookie_merge['name'] == player]['debut'].values[0]+1]
    }

In [58]:
player = players

In [61]:
len(rookie1.name.unique())

312

In [62]:
len(rookie2.name.unique())

312

In [63]:
batting = pd.DataFrame()
pitching = pd.DataFrame()
for p_name in tqdm(list(player.keys())):
    pos = player[p_name]['pos']
    if pos == 'batting':
        no = player[p_name]['p_no']
        year = player[p_name]['year']

        for i in range(2):
            player_df = player_score_batting(year[i], no)
            player_df['name'] = p_name
            player_df['year'] = year[i]
            player_df['p_no'] = no
            player_df['p_pos'] = pos
            player_df['is_debut_year'] = 1 if i == 0 else 0

            batting = pd.concat([batting, player_df])

    else:
        no = player[p_name]['p_no']
        year = player[p_name]['year']

        for i in range(2):
            player_df = player_score_pitching(year[i], no)
            player_df['name'] = p_name
            player_df['year'] = year[i]
            player_df['p_no'] = no
            player_df['p_pos'] = pos
            player_df['is_debut_year'] = 1 if i == 0 else 0

            pitching = pd.concat([pitching, player_df])


100%|██████████| 342/342 [15:44<00:00,  2.76s/it]


In [22]:
batting.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48943 entries, 0 to 55
Data columns (total 37 columns):
 #   Column                                                                       Non-Null Count  Dtype 
---  ------                                                                       --------------  ----- 
 0   Standings Season, 정규 시즌                                                      48943 non-null  object
 1   Other Team, 상대 팀                                                             48943 non-null  object
 2   Game Result, 경기 결과                                                           48943 non-null  object
 3   Batting Order, 타순                                                            48943 non-null  object
 4   Position, 포지션                                                                48943 non-null  object
 5   Games Started, 선발                                                            48943 non-null  object
 6   effective Plate Appearance, 실질타석(유효타석)           

In [64]:
list(batting.columns)

['Standings Season, 정규 시즌',
 'Other Team, 상대 팀',
 'Game Result, 경기 결과',
 'Batting Order, 타순',
 'Position, 포지션',
 'Games Started, 선발',
 'effective Plate Appearance, 실질타석(유효타석)',
 'At Bat, 타수',
 'Runs , 득점',
 'Hits, 안타',
 'Doubles, 2루타',
 'Triples, 3루타',
 'Home runs, 홈런',
 'Total bases, 루타수',
 'Runs batted in, 타점',
 'Stolen bases, 도루 성공',
 'Caught stealing, 도루 실패',
 'Bases on balls, 볼넷',
 'Hit by pitch, 몸에 맞는 볼',
 'Intentional bases on balls, 고의4구',
 'Stirke outs, 삼진',
 'Grounded into double play, 병살타',
 'Sacrifice hits, 희생 번트',
 'Sacrifice flies, 희생 플라이',
 'Batting average, 타율',
 'On-base percentage, 출루율',
 'Sulgging percentage, 장타율',
 'On-base percentage plus slugging, 출루율+장타율',
 'Pitches, 투구수',
 'Average Leverage Index, 영향력 지표 평균',
 'Run Above Average based on the 24 Base/Out States, 베이스/아웃에 따른 기대득점 대비 득점 기여',
 'Win Probability Added, 추가한 승리 확률',
 'name',
 'year',
 'p_no',
 'p_pos',
 'is_debut_year']

In [65]:
batting_rename = {'Standings Season, 정규 시즌': 'SS',
 'Other Team, 상대 팀': 'OT',
 'Game Result, 경기 결과': 'GR',
 'Batting Order, 타순': 'BO',
 'Position, 포지션': "Po",
 'Games Started, 선발': "GS",
 'effective Plate Appearance, 실질타석(유효타석)': "ePA",
 'At Bat, 타수': "AB",
 'Runs , 득점': "R",
 'Hits, 안타': "H",
 'Doubles, 2루타': "D",
 'Triples, 3루타': "T",
 'Home runs, 홈런': "HR",
 'Total bases, 루타수': "Tb",
 'Runs batted in, 타점': "Rb",
 'Stolen bases, 도루 성공': "Sb",
 'Caught stealing, 도루 실패': "Cs",
 'Bases on balls, 볼넷': "Bob",
 'Hit by pitch, 몸에 맞는 볼': "Hbp",
 'Intentional bases on balls, 고의4구': "Ib",
 'Stirke outs, 삼진': "S",
 'Grounded into double play, 병살타': "Gidp",
 'Sacrifice hits, 희생 번트': "Sh",
 'Sacrifice flies, 희생 플라이': "Sf",
 'Batting average, 타율': "Ba",
 'On-base percentage, 출루율': "Op",
 'Sulgging percentage, 장타율': "Sp",
 'On-base percentage plus slugging, 출루율+장타율': "Opp",
 'Pitches, 투구수': "Pit",
 'Average Leverage Index, 영향력 지표 평균': "ALI",
 'Run Above Average based on the 24 Base/Out States, 베이스/아웃에 따른 기대득점 대비 득점 기여':"RAA",
 'Win Probability Added, 추가한 승리 확률': "WPA",
}

In [66]:
batting = batting.rename(columns=batting_rename)

In [67]:
# cleasing data
batting.replace({'': np.nan}, inplace = True)
# play result (W/L : when win, the diff of score)
batting[['Win', 'Scores']] = batting['GR'].apply(lambda x: pd.Series([x[0], x[2:]]))
batting['Win'] = batting['Win'].apply(lambda x: 1 if x == 'W' else 0)
batting['Scores'] = batting['Scores'].apply(lambda x: abs(int(x.split(':')[0]) - int(x.split(':')[1])))
batting.drop('GR', axis = 1, inplace = True)

# play in home or not
batting['home'] = batting['OT'].apply(lambda x:0 if '@' in x else 1)

# concat rookie award year
batting = pd.merge(batting, rookie_merge[['HS_Graduate', 'name','award_year']], on = 'name')
batting['is_award'] = batting['award_year'].apply(lambda x : 1 if pd.isna(x) != 1 else 0)

# convert to datetime
batting['date'] =  pd.to_datetime(batting['year'].astype(str) + '-' + batting['SS'], format='%Y-%m-%d')
batting.drop(['SS','year'], axis = 1,inplace = True)

batting.head()

,OT,BO,Po,GS,ePA,AB,R,H,D,T,...,p_no,p_pos,is_debut_year,Win,Scores,home,HS_Graduate,award_year,is_award,date
0,KIA,8,7,1,4,4,1,1,0,0,...,13168,batting,1,1,1,1,y,2018.0,1,2018-03-24
1,KIA,8,7,1,4,4,1,1,0,0,...,13168,batting,1,1,1,1,y,NaN,0,2018-03-24
2,KIA,8,7,1,3,3,0,1,0,0,...,13168,batting,1,0,13,1,y,2018.0,1,2018-03-25
3,KIA,8,7,1,3,3,0,1,0,0,...,13168,batting,1,0,13,1,y,NaN,0,2018-03-25
4,SK,8,10,1,4,3,2,2,0,0,...,13168,batting,1,0,3,1,y,2018.0,1,2018-03-27


In [68]:
rookie.head()

,award_year,name,club,position,debut,num,HS_Graduate
0,2023,문동주,한화,pitching,2022,15013,y
1,2022,정철원,두산,pitching,2018,13071,y
2,2021,이의리,KIA,pitching,2021,14788,y
3,2020,소형준,KT,pitching,2020,14600,y
4,2019,정우영,LG,pitching,2019,14136,y


In [69]:
batting.columns

Index(['OT', 'BO', 'Po', 'GS', 'ePA', 'AB', 'R', 'H', 'D', 'T', 'HR', 'Tb',
       'Rb', 'Sb', 'Cs', 'Bob', 'Hbp', 'Ib', 'S', 'Gidp', 'Sh', 'Sf', 'Ba',
       'Op', 'Sp', 'Opp', 'Pit', 'ALI', 'RAA', 'WPA', 'name', 'p_no', 'p_pos',
       'is_debut_year', 'Win', 'Scores', 'home', 'HS_Graduate', 'award_year',
       'is_award', 'date'],
      dtype='object')

In [70]:
len(batting.columns)

41

In [71]:
batting.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98403 entries, 0 to 98402
Data columns (total 41 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   OT             98403 non-null  object        
 1   BO             98391 non-null  object        
 2   Po             98391 non-null  object        
 3   GS             98403 non-null  object        
 4   ePA            98403 non-null  object        
 5   AB             98403 non-null  object        
 6   R              98403 non-null  object        
 7   H              98403 non-null  object        
 8   D              98403 non-null  object        
 9   T              98403 non-null  object        
 10  HR             98403 non-null  object        
 11  Tb             98403 non-null  object        
 12  Rb             98403 non-null  object        
 13  Sb             98403 non-null  object        
 14  Cs             98403 non-null  object        
 15  Bob            9840

In [72]:
batting = batting.dropna(subset=['BO','Po'])

In [73]:
for i in batting.columns:
    print(f'{i}: {sum(batting[i].isna())}')

OT: 0
BO: 0
Po: 0
GS: 0
ePA: 0
AB: 0
R: 0
H: 0
D: 0
T: 0
HR: 0
Tb: 0
Rb: 0
Sb: 0
Cs: 0
Bob: 0
Hbp: 0
Ib: 0
S: 0
Gidp: 0
Sh: 0
Sf: 0
Ba: 0
Op: 0
Sp: 0
Opp: 0
Pit: 0
ALI: 43190
RAA: 43190
WPA: 43190
name: 0
p_no: 0
p_pos: 0
is_debut_year: 0
Win: 0
Scores: 0
home: 0
HS_Graduate: 0
award_year: 95463
is_award: 0
date: 0


In [74]:
batting = batting[batting['BO'] != 'P'] # 장영석 BO타순에 P가 있음 한개 밖에 없으므로 제거

In [75]:
for i in ['BO', 'Po', 'GS', 'ePA', 'AB', 'R', 'H', 'D', 'T', 'HR','Tb', 'Rb', 'Sb', 'Cs', 'Bob', 'Hbp', 'Ib', 'S', 'Gidp', 'Sh', 'Sf']:
       print(i)
       batting[i] = batting[i].astype(dtype = "int64")

for i in ['Ba', 'Op', 'Sp', 'Opp', 'Pit', 'ALI', 'RAA', 'WPA']:
    batting[i] =  batting[i].astype(float)

BO
Po
GS
ePA
AB
R
H
D
T
HR
Tb
Rb
Sb
Cs
Bob
Hbp
Ib
S
Gidp
Sh
Sf


In [76]:
batting.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 98389 entries, 0 to 98402
Data columns (total 41 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   OT             98389 non-null  object        
 1   BO             98389 non-null  int64         
 2   Po             98389 non-null  int64         
 3   GS             98389 non-null  int64         
 4   ePA            98389 non-null  int64         
 5   AB             98389 non-null  int64         
 6   R              98389 non-null  int64         
 7   H              98389 non-null  int64         
 8   D              98389 non-null  int64         
 9   T              98389 non-null  int64         
 10  HR             98389 non-null  int64         
 11  Tb             98389 non-null  int64         
 12  Rb             98389 non-null  int64         
 13  Sb             98389 non-null  int64         
 14  Cs             98389 non-null  int64         
 15  Bob            9838

In [77]:
# 없는 지표값들이 0으로 바꿔도 되는 데이터임
batting = batting.fillna(0)

In [46]:
batting.to_csv('batting.csv', sep=',', na_rep='NaN')

## 투수 데이터 정제

In [ ]:

pitching.info() # 투수가 별로 없넹...

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1683 entries, 0 to 40
Data columns (total 34 columns):
 #   Column                                                                       Non-Null Count  Dtype 
---  ------                                                                       --------------  ----- 
 0   Standings Season, 정규 시즌                                                      1683 non-null   object
 1   Other Team, 상대 팀                                                             1683 non-null   object
 2   Game Result, 경기 결과                                                           1683 non-null   object
 3   Games Started, 선발                                                            1683 non-null   object
 4   Innings pitched, 이닝                                                          1683 non-null   object
 5   Runs, 실점                                                                     1683 non-null   object
 6   Earned runs, 자책점                                  

In [ ]:
batting_rename = {'Standings Season, 정규 시즌': 'SS',
 'Other Team, 상대 팀': 'OT',
 'Game Result, 경기 결과': 'GR',
 'Batting Order, 타순': 'BO',
 'Position, 포지션': "Po",
 'Games Started, 선발': "GS",
 'effective Plate Appearance, 실질타석(유효타석)': "ePA",
 'At Bat, 타수': "AB",
 'Runs , 득점': "R",
 'Hits, 안타': "H",
 'Doubles, 2루타': "D",
 'Triples, 3루타': "T",
 'Home runs, 홈런': "HR",
 'Total bases, 루타수': "Tb",
 'Runs batted in, 타점': "Rb",
 'Stolen bases, 도루 성공': "Sb",
 'Caught stealing, 도루 실패': "Cs",
 'Bases on balls, 볼넷': "Bob",
 'Hit by pitch, 몸에 맞는 볼': "Hbp",
 'Intentional bases on balls, 고의4구': "Ib",
 'Stirke outs, 삼진': "S",
 'Grounded into double play, 병살타': "Gidp",
 'Sacrifice hits, 희생 번트': "Sh",
 'Sacrifice flies, 희생 플라이': "Sf",
 'Batting average, 타율': "Ba",
 'On-base percentage, 출루율': "Op",
 'Sulgging percentage, 장타율': "Sp",
 'On-base percentage plus slugging, 출루율+장타율': "Opp",
 'Pitches, 투구수': "Pit",
 'Average Leverage Index, 영향력 지표 평균': "ALI",
 'Run Above Average based on the 24 Base/Out States, 베이스/아웃에 따른 기대득점 대비 득점 기여':"RAA",
 'Win Probability Added, 추가한 승리 확률': "WPA",
}